In [1]:
import os
import json
import itertools

## ScienceQA - Chameleon GPT-4

In [2]:
result_file = "../../results/scienceqa/chameleon_gpt4_test_cache.jsonl"
data_file = "../../data/scienceqa/problems.json"

prob_data = json.load(open(data_file))

with open(result_file) as f:
    results = {}
    for line in f:
        try:
            data = json.loads(line)
            results[data["pid"]] = data
        except:
            pass

In [3]:
modules_list = []

for pid, result in results.items():
    modules = result["modules:output"]
    if "program_generator_and_verifier" in modules:
        index = modules.index("program_generator_and_verifier")
        nested = []
        for module in modules:
            if module == "program_generator_and_verifier":
                nested.append(["program_generator", "program_verifier"])
            else:
                nested.append([module])
        modules = [m for sublist in nested for m in sublist]
        #print(modules)
    
    modules = ["START"] + modules + ["END"]
    modules_list.append(modules)

In [4]:
# modules_list

In [5]:
transition_num = {
 "START": {},
 'knowledge_retrieval': {},
 'text_detector': {},
 'query_generator': {},
 'bing_search': {},
 'image_captioner': {},
  'solution_generator': {},
 'answer_generator': {},
}


transitions = {
 "START": {},
 'knowledge_retrieval': {},
 'text_detector': {},
 'query_generator': {},
 'bing_search': {},
 'image_captioner': {},
  'solution_generator': {},
 'answer_generator': {},
}

In [6]:
for modules in modules_list:
    for i, module in enumerate(modules[:-1]):
        next_module = modules[i+1]
        if next_module not in transition_num[module]:
            transition_num[module][next_module] = 1
        else:
            transition_num[module][next_module] += 1
            
        transitions[module][next_module] = round(transition_num[module][next_module] / sum(transition_num[module].values()), 2)

In [7]:
transition_num

{'START': {'knowledge_retrieval': 2241,
  'text_detector': 1257,
  'query_generator': 471,
  'image_captioner': 272},
 'knowledge_retrieval': {'solution_generator': 3443, 'bing_search': 5},
 'text_detector': {'knowledge_retrieval': 981, 'solution_generator': 276},
 'query_generator': {'bing_search': 473},
 'bing_search': {'solution_generator': 465,
  'knowledge_retrieval': 10,
  'image_captioner': 3},
 'image_captioner': {'knowledge_retrieval': 216,
  'solution_generator': 57,
  'query_generator': 2},
 'solution_generator': {'answer_generator': 4241},
 'answer_generator': {'END': 4241}}

In [8]:
transitions

{'START': {'knowledge_retrieval': 0.53,
  'text_detector': 0.3,
  'query_generator': 0.11,
  'image_captioner': 0.06},
 'knowledge_retrieval': {'solution_generator': 1.0, 'bing_search': 0.0},
 'text_detector': {'knowledge_retrieval': 0.78, 'solution_generator': 0.22},
 'query_generator': {'bing_search': 1.0},
 'bing_search': {'solution_generator': 0.97,
  'knowledge_retrieval': 0.02,
  'image_captioner': 0.01},
 'image_captioner': {'knowledge_retrieval': 0.79,
  'solution_generator': 0.21,
  'query_generator': 0.01},
 'solution_generator': {'answer_generator': 1.0},
 'answer_generator': {'END': 1.0}}

In [9]:
for module in transitions:
    print(module.replace("_", " "))
    
    next_module = transitions[module]
    for m in next_module:
        print(f" -> {m}: {next_module[m]}".replace("_", " "))
        
    print("")

START
 -> knowledge retrieval: 0.53
 -> text detector: 0.3
 -> query generator: 0.11
 -> image captioner: 0.06

knowledge retrieval
 -> solution generator: 1.0
 -> bing search: 0.0

text detector
 -> knowledge retrieval: 0.78
 -> solution generator: 0.22

query generator
 -> bing search: 1.0

bing search
 -> solution generator: 0.97
 -> knowledge retrieval: 0.02
 -> image captioner: 0.01

image captioner
 -> knowledge retrieval: 0.79
 -> solution generator: 0.21
 -> query generator: 0.01

solution generator
 -> answer generator: 1.0

answer generator
 -> END: 1.0

